In [1]:
import re
import string
import datetime
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans

In [ ]:
#imported all the required packages

In [2]:
try:
    data = open("./foods.txt","r",encoding='latin-1')
except IOError:
    print("Could not open file!")
longWordFile = open ("Long Word List.txt","r")

In [ ]:
#reading the food dataset

In [3]:
fileStr = ''
for line in data:
    if line[0:13] == 'review/text: ':
        fileStr = fileStr + line[13:]
fileStr = fileStr.lower()

In [ ]:
#reading the review/text: field data from the food.txt file and converting into lowercase

In [4]:
fileStr_replacepunct = fileStr.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
fileStr_minusgarbage = re.sub('\d+|br ', ' ', fileStr_replacepunct)
fileStr_minusspaces = re.sub(' +', ' ', fileStr_minusgarbage)
fileStr_filtered = re.sub('\s+', ' ', fileStr_minusspaces)

In [ ]:
#preprocessing using regular expression to remove spaces, puntctuations, other unwated components

In [5]:
fileList = list(fileStr_filtered.strip().split(' '))

In [ ]:
#spliting the string on every ' ' and storing in a list

In [6]:
L = set(fileList)

In [ ]:
#requirement part 1

In [7]:
wordStr = ''
for word in longWordFile:
    wordStr = wordStr + word
wordStr = wordStr.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
wordStr = re.sub('\s+', ' ', wordStr)

In [ ]:
#Removing stop words using the long word list from the rank.nl website

In [8]:
wordList = list(wordStr.split(' '))

In [ ]:
#word list of stop words

In [9]:
W = L.difference(set(wordList))

In [ ]:
#Word list after filteration part 2 requirement 

In [ ]:
countDict = {x:fileList.count(x) for x in W}
print("length of L",len(L))
print("length of countDict",len(countDict))


In [ ]:
sortedDict = sorted(countDict.items(), key = lambda x : x[1],reverse=True)

In [ ]:
# reverse sorting dictionary (descending order of words with their freq.) 

In [ ]:
top500Words = dict(sortedDict[:500])  

In [ ]:
#taking the top 500 words

In [ ]:
with open("top_500_words.txt", "w") as outputFile:
    for k, v in top500Words.items():
        top500Words_content =" ("+ k + ", " + str(v) + "),"
        outputFile.write(top500Words_content)

In [ ]:
#storing output

In [ ]:
corpus = list(top500Words.keys())
reviewList_fordataframe = fileStr_minusspaces.split('\n')
dataFrame = pd.DataFrame(reviewList_fordataframe)
vectorObj = dataFrame.iloc[0:,0]
vectorizer = CountVectorizer()
vectorizer.fit(corpus)
X = vectorizer.transform(vectorObj)
sparseMatrix = X.A

In [ ]:
#vectorize using 500 words

In [ ]:
k = 10
model = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=1)
model.fit(sparseMatrix)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind])


In [ ]:
#applying kmeans 

In [ ]:
testFile.close()
longWordFile.close()